# Data reduction and analysis magic with `Pandas`

## Recap

In the previous lesson we covered some of the basics of `pandas`:
- creating dataframes from series(es)
- writing a dataframe to file
- reading a dataframe from file
- accessing rows and columns within a dataframe
- accessing subsets of data within a dataframe based on values meeting some conditions
- creating new columns in a dataframe 

If you're like most people, you're probably thinking "Wow, that is all very fancy (*i.e.*, confusing), but I'll stick with my Excel spreadsheets, k thx bye."  In this lesson, we'll see some of the true magic of `pandas` that makes it (or something like it) an essential tool for meaningful data analysis.

***

## Motivation

Data analysis is an incredibly powerful way of knowing things.  In general, the larger a dataset is (the more "robust" it is), the more confident we can be that the results derived from that dataset are significant.  We want to be confident that our results reflect general features fo the system we're studying and not some weird statistical fluctuation, and *big* datasets are generally the way to achieve this.  

For example, it would be a bad idea to study the average height of human beings by looking at a dataset containing the heights of only the 5 starting players on the 2023 Denver Nuggests basketball team.  Big datasets are important for making sure that we're doing careful science.

That said, quanttative data in large amounts can be difficult for us humans to process.  If you opened a spreadsheet containing, say, 5 rows of human height data you might be able to glance through the individual heights, ballpark the average height at 6'8", and determine that something is really funky with the data.  ("Are these people all pro basketbal players?")  If, however, you opened a spreadsheet containing height measurements for all of the 12.96 million residents of the state of Pennsylvania your eyes might glaze over in panic and you might just trust whatever results a researcher derived from the data.  Large amounts of data are simply overwhelming to our squishy human brains.

A beautiful illustration of this is the 2021 scientific misconduct scandal surrounding Dan Ariely, a Duke University Professor of Psychology and popular science author.  Ariely rose to national attention due in part to really good-sounding "results" from studies of human behavior, many of which focused on honesty.  In 2021, a team of data sleuths at Data Colada uncovered rather blatant data fabrication and manipulation in Ariely's work.  Ariely's subterfuge relied on the assumption that the dataset he was working with was too big for anyone to ever want to check his math.  Oops!

There are many great articles about this online.  I strongly recommend learning more about this, as it's a great example of how not to do science.
- Planet Money. [Did two honesty researchers fabricate their data?](https://open.spotify.com/episode/2Bse6AcwmJdHRE1hesMUxD?si=lWZfomnqQS-GyhtEPvj1-A)
- Data Colada. [Evidence of Fraud in an Influential Field Experiment About Dishonesty](https://datacolada.org/98)
- The New Yorker. [They Studied Dishonesty. Was Their Work a Lie?](https://www.newyorker.com/magazine/2023/10/09/they-studied-dishonesty-was-their-work-a-lie)

***

## Data reduction

[Wikipedia](https://en.wikipedia.org/wiki/Data_reduction) tells us that 

> **Data reduction** is the transformation of numerical or alphabetical digital information derived empirically or experimentally into a corrected, ordered, and simplified form. The purpose of data reduction can be two-fold: reduce the number of data records by eliminating invalid data or *produce summary data and statistics at different aggregation levels for various applications*.

The italicized text is what we're after here.  In previous lessons, we've learned how to do things like fit models to data to generate results.  Data reduction is the process that happens before that.  Data reduction often means taking a large amount of raw data and collapsing, massaging, and processing it into a smaller set of data from which we can generate results.  The term "data reduction" seems to have been replaced with the more general "data analysis," but it's good to know what reduction is a particular step in many scientific processes.

`Pandas` is an amazing tool for data reduction.  It can do quite easily some things that MS Excel (and similar) would just cry about.  We'll investigate these capabilities with a fairly easy-to-understand example.

### Netflix data

On 12 Dec 2023 Netflix [broke its silence](https://about.netflix.com/en/news/what-we-watched-a-netflix-engagement-report) about the popularity of content on its platform between.  Prior to this, Netflix would rank its top performers, but wouldn't release any absolute data on how much people watched each show/movie what it calls "engagement".  The report that Netflix released gives viewership information for titles between January and June of 2023.  A `csv` file of this data exists in the `data_files/` directory.  Let's load it up!

In [ ]:
import pandas as pd
import numpy as np

netflix_df = pd.read_csv('data_files/netflix_engagement_report_2023jan-jun.csv')

This is a fairly big dataset:

In [ ]:
display(netflix_df.head(10))
display(netflix_df.tail(10))

print(netflix_df.shape)

OK, so the dataframe contains title, availability, premier/release date, and hours watched for... 18,214 titles!  Yikes!  It also looks like the titles are ordered by the date that they became available, which is perhaps not super convenient.  It **is** nice to see *Arrested Development*, though!  Note that there are titles with non-Enlgish characters in them -- cool!  There are also titles that don't have a release date; `NaN` is pythonic for 'not a number'.

You can maybe guess why a dataset like this might be very helpful to a company.  If they can determine features of the most-watched shows and find a way to make more of the same for less money, they can make ~~more crap that maximizes shareholder value~~ art that helps us understand what it means to be human.  Still, though, we need to do some analysis on this dataset in order to extract meaning/understanding from it.

***

### What do we want to know?

At this point, I think that it will be helpful to establish a few research questions that we can investigate with this dataset.  

1. What was the most-viewed title during this period?
2. What's more popular, TV shows or movies?
3. What was the total watch time for the titles in this dataset?
4. How many unique properties are there in the dataset?  (*I.e.*, multiple seasons of the same show count as a single property.)
5. **Brainstorm as a group and add your questions here!**

What we want to know will guide which tools we use.

***

### `apply`

#### `apply` example 1

First, you may notice that the Hours Viewed column is a total pain -- Netflix has presented these large numbers with commas in them, which means that they appear in the dataframe as strings.  We need to fix this by producing a column of numerical values.

Let's begin by writing a function which converts a single string such as this to an integer:

In [ ]:
def convert_bad_number(s):
    s = s.replace(',','') # replaces commas with nothing
    n = int(s) / 1e6      # unit conversion, divides by 1 million
    return n

convert_bad_number('1,234,567') # let's test this function

Great!  It would be really convenient if we could just apply this to an entire column of the dataframe to produce a new column.  Let's test this out in the cell below (uncomment the code and run the cell):

In [ ]:
# netflix_df['megahours_viewed'] = convert_bad_number(netflix_df['Hours Viewed'])

Oh nuts!  Well, maybe we should have expected that.  

If the data were stored in something like a list, we would probably write a loop over the list to apply our function to the elements in succession.  This is possible with a dataframe but it is *incredibly* slow, so I'm not even going to show you how to do it.

One **good** way to apply a function to each element of a column in a dataframe is to use the `Dataframe.apply` function.  We'll create the new column by `apply`-ing our function to the appropriate column:

In [ ]:
netflix_df['megahours_viewed'] = netflix_df['Hours Viewed'].apply(convert_bad_number)

display(netflix_df.head(10))

Hey, looking good!

I should mention quickly that, while this is a valid way to do this, it is usually not the computationally *quickest* way to do it.  But it's fine for our purposes.


#### `apply` example 2

The release date is also pretty interesting, but it's a bit difficult to work with.  Let's try to make a column that gives the number of days between the release date and the end of the viewing period (30 June 2023).  We'll do the same type of thing that we did above: write a function and `apply` it to a column.

I'm going to use the `datetime` module to speed things up.  If you haven't worked with `datetime` before, that's fine.  Read through the function and see if you can figure how it works.  Note that we have to build in a conditional for those titles that don't have a release date.

In [ ]:
from datetime import date

def calc_age(release_date):
    if pd.isna(release_date): # if the release date is NaN
        return None
    else:
        release_date = release_date.split('-') # split the string
        release_date = date(int(release_date[0]), int(release_date[1]), int(release_date[2]))
        end_date = date(2023, 6, 30) # set the end date
        delta = end_date - release_date # calculate the difference between dates
        return(delta.days + 1) # add 1 so that we don't divide by zero later!

Now let's apply this function to the appropriate column in the dataframe.

In [ ]:
netflix_df['age_days'] = netflix_df['Release Date'].apply(calc_age)

display(netflix_df.head(10))
display(netflix_df.tail(10))

#### `apply` example 3

Many of the titles appear to be series, or what old people like me call "TV shows".  We may wish to know how many different "properties" there are in this dataset.  *I.e.*, a movie like *Lost in Translation* is a single artistic entity, whereas *Trailer Park Boys: Season 6* is simply another installment in an artistic entity.  

Let's write a function that grabs all of the stuff before the colon (":") in a title an puts it in a new column called `property`.

In [ ]:
def property(t):
    prop = t.split(":")[0]
    return prop

netflix_df['property'] = netflix_df['Title'].apply(property)

display(netflix_df.head(10))

Looks like it worked well enough!  Whether we can reliably interpret this new column is something that we'll need to investigate later on.  For example, I'm not sure what to make of "Kevin Hart: What Now?"

#### Warm-up problem (must do this!)

Below is a function that determines whether 'Season' or 'Series' appears in a string.  Write some code that applies this function to the titles in the dataframe to produce a new column called 'series?'.  Later we'll use this to approximate whether each title is more like a TV show or a movie.

In [ ]:
def season_in_title(t):
    if 'Season' in t or 'Series' in t:
        return 1
    else:
        return 0

In [ ]:
#### your code goes here



display(netflix_df.head(20))

#### Warm-up problem

Below is a function that counts the number of non-Roman characters in a string.  Write some code that applies this function to the titles in the dataframe to produce a new column called 'non_roman_characters'.

In [ ]:
roman_chars = list('abcdcefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ 1234567890!.,;:?')
def get_non_roman_chars(s):
    non_roman_chars = 0
    s_arr = list(s)
    for c in s_arr:
        if c not in roman_chars:
            non_roman_chars += 1
    return non_roman_chars

In [ ]:
#### your code goes here


display(netflix_df.head(10))
display(netflix_df.tail(10))

#### Warm-up problem

Now you try!  Write a function, `bool_convert`, that takes a string as input and produces a 0 if the string is 'No' and a 1 if the string is 'Yes'.  Apply this function to the "Available Globally?" column of the dataframe to produce a new column called "avail_globally_int".

In [ ]:
#### your code goes here

***

## Re-ordering the dataset --> `sort_values`

So far we've been taking glances at the dataframe by using the `head` and `tail` functions.  But the dataframe is ordered based on titles' release dates, which isn't super helpful for many of the research questions we established above.  

The `Dataframe.sort_values` function can re-order the dataframe according to any column(s) that we provide.  For example, we can answer our first research question by doing the following:

In [ ]:
netflix_df = netflix_df.sort_values('megahours_viewed')

display(netflix_df.head(10))

You'll notice that it looks like `pandas` defaults to ascending order; it put the least-viewed titles at the top of the dataframe.  We can reverse this by setting the `ascending` keyword to `False`:

In [ ]:
netflix_df = netflix_df.sort_values('megahours_viewed', ascending=False)

display(netflix_df.head(20))

So there's the answer to our first question!  *The Night Agent* (which I am just now learning is a thing that exists) was the most-viewed show during this period.

Notice that the index column on the left still shows the original indices of the rows in the dataframe.  If we're confident that we'd like to keep this new ordering of the dataframe, we can reset the indices thusly:

In [ ]:
netflix_df = netflix_df.reset_index(drop=True)

display(netflix_df.head(20))

Congratulations, *The Night Agent* -- you're #0!

The majority of the top-twenty titles appear to be what we could consider to be shows/series!  This is maybe not surprising.  

`sort_values` also allows you to sort based on two features.  Say we want to sort based on the days since release and then by the number of hours viewed.  We supply `sort_values` with a list of column names.  I just want to show you this; I don't want to actually change the dataframe.  See if you can parse the code below:

In [ ]:
display(netflix_df.sort_values(['age_days', 'megahours_viewed']).head(20))

***

## Data reduction 

OK, it feels like we're getting a handle on what's in this dataset.  We can now try to *reduce* it, to boil its contents down to gain some higher-level understanding.

Note that we can generate summary statistics for the entire dataset using basic `pandas` functions on the individual columns.  These summary statistics reduce the data *all the way down* to a small set of values.

Before you continue, please interpret the results of each of the following lines of code (*i.e.*, talk with the person sitting next to you about what each of the output values means).  You'll may have to Google the last one; that's ok.

In [ ]:
print(netflix_df['megahours_viewed'].max())
print(netflix_df['megahours_viewed'].sum())
print(netflix_df['megahours_viewed'].median())
print(netflix_df['megahours_viewed'].mean())
print(netflix_df['megahours_viewed'].std())
print('- - - - -')
print(netflix_df['megahours_viewed'].quantile([0.25, 0.5, 0.75, 1.0]))

### `groupby`

If you're a data science exec at Neflix, a lot of your job is probably helping the company to determine what to make and market next to ~~increase viewership or subsciptions~~ investigate the human condition.  Said simply: What do the people want?  (Or what can you convince them that they want?)  The summary statistics above are not sufficient to answer this question because in calculating them, we've reduced out all of the information about any groups of titles.

Luckily, the `Dataframe.groupby` function in `pandas` can  do what we want!  

Let's first look at the properties that are series.  These seem to do well.  I'll create a dataset that includes only those titles that have a 1 for the `series?` feature.  Note that if you didn't complete the first warm-up problem, the cell below **will not work!**

In [ ]:
series_df = netflix_df[netflix_df['series?'] == 1]
display(series_df.head(10))

Now let's use `groupby` to group titles that are of the same property.  In general, `groupby` combine all of the rows in a dataframe that have the same value for the column(s) that you specify.  For example, doing a `groupby` on `property` will group all of the seasons of *The Unbreakable Kimmy Schmidt* together as a single entry (and similar for other properties).

When we group entries in this way, there will be some information that we want to keep and some that we will discard.  For example, we almost certainly want to keep the hours watched information (to either sum or average), but we will likely discard the release dates since there isn't a sensible way to combine the multiple release dates for the different seasons of a show. 

We have have to tell `pandas` which information to keep (*i.e.*, `megahours_viewed`) and how we want to combine it for the various entries.  Let's start by simply adding all of the `megahours_viewed` for the entries that we group:

In [ ]:
series_df_0 = netflix_df[netflix_df['series?'] == 1]

series_df_0 = series_df_0.groupby('property')['megahours_viewed'].sum().reset_index()
series_df_0 = series_df_0.sort_values('megahours_viewed', ascending=False)
display(series_df_0.head(20))

Next we'll count number of titles that are grouped to each property:

In [ ]:
series_df_1 = netflix_df[netflix_df['series?'] == 1]

series_df_1 = series_df_1.groupby('property')['Title'].count().reset_index()
series_df_1 = series_df_1.sort_values('Title', ascending=False)
display(series_df_1.head(20))

... which confirms my suspicion that *Grey's Anatomy* has been on for tooooooo loooooooong.

This looks good so far, but it's also inefficient.  What if I wanted to both count the seasons *and* sum the hours viewed?  I don't want to have to carry two separate dataframes around.

### `agg`

The `Dataframe.agg` function allows us to give multiple reduction instructions to `groupby`.  `agg` is short for 
> **aggregate** (/ˈaɡ rə ɡət/) *adj.*, formed or calculated by the combination of many separate units or items

With `agg`, we supply a **dictionary** (you remember those, right?) of feature/function pairs.  For exmaple, I can do both of the reductions above with the following:

In [ ]:
series_df = netflix_df[netflix_df['series?'] == 1]

agg_dict = {'Title': 'count', 'megahours_viewed': 'sum'}

series_df = series_df.groupby('property').agg(agg_dict).reset_index()
series_df = series_df.sort_values('Title', ascending=False)
display(series_df.head(20))

One weirdness of `agg` is that we supply *strings* of the function names!  So in the above the function named `'sum'` gets applied to the `'megahours_viewed'` column, and `count` counts the unique entries in the `Title` column.

Check out what we can now do with this reduced dataset!  We'll rename one of the columns and then calculate the average viewership per season:

In [ ]:
series_df = series_df.rename(columns={'Title': 'n_seasons'})

series_df['Mhv_per_season'] = series_df['megahours_viewed'] / series_df['n_seasons']

display(series_df.sort_values('Mhv_per_season', ascending=False).head(20))

Wow, *The Night Agent* (which is totally a thing that I knew about for several notebook cells now) wins again!

***

## What about science, though?

I know what you're thinking: the above examples with Netflix data were really, really, really great, but I'm trying to be a scientist over here!  How can these skills help me do science?

Let's turn our attention to a large (but not nearly the whole thing) dataset from the [Sloan Digital Sky Survey](https://en.wikipedia.org/wiki/Sloan_Digital_Sky_Survey) (SDSS).  Please click that link to read through the SDSS Wikipedia entry -- it is an amazing instrument that has led to some excellent science!  The Tl;dr is that SDSS is a telescope in New Mexico that has taken spectra of over $10^6$ astronomical objects (stars, galaxies, quasars).  You're now going to analyze some of that data!  

A quick aside about how we got the data you're about to analyze.  SDSS makes all of its data available to the public.  Because the data is so voluminous (many, many objects) and complicated (position information, spectra, optical images) there is a sophisticated (SQL) web interface called SkyServer that is used to select and download subsets of the data.  You can find SkyServer [here](https://skyserver.sdss.org/dr16/en/tools/search/sql.aspx). The SQL codee that I used to generate the dataset below is:
```
SELECT TOP 100000
   s.specobjid, s.class, s.z as redshift,
   s.plate, s.mjd, s.fiberid, s.ra, s.dec,
   s.fiberid, s.zErr, s.zWarning,
   s.subClass, s.velDisp, s.velDispErr,
   s.spectroFlux_u, s.spectroFlux_g, s.spectroFlux_r, s.spectroFlux_i, s.spectroFlux_z,
   s.elodieObject, s.elodieSpType, s.elodieBV
FROM SpecObj as s
```
You do **not** need to know any of this, but I'm including it here for completeness.

Another quick aside just to brag.  I (Mike McCracken) have several SDSS spectrometer plates in my office!  Stop by to see them!

You'll find the resulting data file in `data_files/` (duh).  This is the real stuff, people!  Let's check it out:

In [ ]:
sdss_df = pd.read_csv('data_files/SDSS_Skyserver_SQL1_2_2024_100kObjs_SpecObj.csv')
print(sdss_df.shape)
print(sdss_df.columns)
display(sdss_df.head(10))

Wow!  So this dataset contains a 22 features for $1\times10^5$ astronomical objects studied by SDSS!  (Note: You can use SkyServer to generate a file with up to $5\times 10^5$ objects if you're feeling frisky.) 

For now we're going to focus on only a few of the features:
- `specobjid` is a unique numerical identifier for each object in the dataset.  
- `class` is a string that tells us the basic type of the object (star, galaxy, QSO).  You should Google what a 'QSO' is.
- `subClass` is a string that gives us some more information about what the object is.
- `redshift` is a measure of the amount by which the light in the spectrum from that object is redshifted. This quantity is given the symbol $z$ in astronomy.  More below.
- `zErr` is the uncertainty in the measured redshift for the object.

#### Astronomy crash-course (from a particle physicist)

Here's a really basic idea of how this data was collected.  Scientists point SDSS at a particular object in the sky.  They collect a spectrum of the light that the object emits.  Based on the features of the spectrum, they determine the class and subclass of the object.

*Redshift* is a quantity that is essential to astronomy.  The basics are the following.  Special relativity tells us that the light emitted by an object moving away from an observer will will be shifted to longer wavelengths.  For example, stars emit light at all wavelengths in the visible spectrum.  The light from a star moving away from us will be shifted to longer wavelengths.  Red is on the long-wavelength end of the visible spectrum, so we call this shifting 'redshift'.  One can determine the redshift of, say, a star through the following process:
1. Choose a known process that is likely to happen in the star.  A good choice would be something like one of the balmer-series transitions in hydrogen.
2. Measure the wavelength of this transition in a laboratory on Earth.  For example, the $n=4$ to $n=2$ transition in hydrogen produces a photon with wavelength $\lambda_{emit} = 486.135\textrm{ nm}$.
3. Measure the wavelength of the corresponding feature in the *observed* spectrum from the star (likely by fitting).  Call this value $\lambda_{obs}$.
4. Calculate the redshift as $z = (\lambda_{obs} - \lambda_{emit})/\lambda_{emit}$.

The larger the redshift ($z$), the more difference there is between the emitted and observed wavelengths.  The larger the difference between the wavelengths, the faster the object is receding (moving away) from us (the observer).  

#### Mean $z$ values for different classes of objects

Let's do some grouping of this data to see if we can determine any significant differences in redshift between the different types of object.

To do this, I want to show off another way to give instructions to the `agg` function.  See if you can follow.

In [ ]:
sdss_df = pd.read_csv('data_files/SDSS_Skyserver_SQL1_2_2024_100kObjs_SpecObj.csv')

sdss_df = sdss_df[sdss_df['zWarning'] == 0] # this removes objects with some suspicious features

def negative_z(z):
    if z < 0:
        return 1
    else:
        return 0

sdss_df['negative_redshift'] = sdss_df['redshift'].apply(negative_z)

sdss_df = sdss_df.groupby(['class']).agg(n_objects=('specobjid', 'count'),
                                         redshift_mean=('redshift', 'mean'),
                                         redshift_std=('redshift', 'std'),
                                         redshift_min=('redshift', 'min'),
                                         redshift_max=('redshift', 'max'),
                                         n_negative_redshift=('negative_redshift', 'sum'),
                                         zErr_mean=('zErr', 'mean')).reset_index()

display(sdss_df.sort_values('redshift_mean'))

#### It's science time!

We see very different values for the redshifts of stars, galaxies, and quasars (oops, I told you what 'QSO' means).  Below are a few questions that we can investigate with this reduced data.  You should answer at least the first two (talk with your neighbors) before you move on.

1. (Regular) Why is the mean redshift for most stars negative?  What does this mean about stars' motion relative to us?  Why would this number on average be negative (that is, what's the physical reason for this)?
2. (Semi-meanie) Nearly all 66.8 thousand of the galaxies in the dataset have positive $z$, meaning that they are moving away from us.  Why?  If you can't figure this out, don't feel bad -- Einsten (yes, that Einstein) [couldn't either](https://bigthink.com/13-8/lemaitre-priest-proved-einstein-wrong/).
3. (Meanie) By now you should have Googled what a quasar ('QSO') is.  Why do we only observe them with large redshifts?  Here's a hint: For the largest-$z$ quasar in the dataset ($z = 6.99$), the 'lookback time' is approximately $13.0$ billion years.  (I'm not going to tell you what those words mean.)

You may choose to work with this dataset more in the homework, but for now I'll simply point out that each of the object classes contains a number of subclasses, for a total of 52 subclasses!

In [ ]:
sdss_df = pd.read_csv('data_files/SDSS_Skyserver_SQL1_2_2024_100kObjs_SpecObj.csv')
sdss_df = sdss_df[sdss_df['zWarning'] == 0]
sdss_df = sdss_df.groupby(['class', 'subClass']).agg(n_objects=('specobjid', 'count'),
                                         redshift_mean=('redshift', 'mean'),
                                         redshift_std=('redshift', 'std'),
                                         redshift_min=('redshift', 'min'),
                                         redshift_max=('redshift', 'max'),
                                         zErr_mean=('zErr', 'mean')).reset_index()

display(sdss_df.sort_values('redshift_mean'))
print(sdss_df.shape)

***

## In summary

Perhaps you're an Excel expert (Chem majors, are you with me?) and during the previous lesson on `pandas` basics you thought that this approach is just too much work get you to move away from a pointy-clicky spreadsheet program.
Today's lesson should convince you that there are some (important) ways of working with data that Excel is just not suited for.  The aggregation of the SDSS data above would have taken approximately 13.0 billion years to do in Excel, and that's assuming that Excel didn't crash when you tried to open a csv file that big (and that you paid for Excel in the first place).

The tools that we've investigated above allow us to ask and answer new types of questions with data!  Hopefully they encourage you to think about science in a different way. 

***

## Problems 

Your homework is to complete either (three of the numbered problems) or (the single Meanie).

### Regulars

#### Problem 1
Using the Netlfix dataset, determine the titles with top-ten highest 'freshness'.  'freshness' is defined (by me) as the hours watched divided by the 'age' of the show in days.  

#### Problem 2
The Netflix dataset gives the hours watched during the first half of 2023.  Find the top-ten most-watched (duirng this period) titles that were not released during this period.

#### Problem 3
What fraction of Netlfix titles are available globally?  Is there is a significant difference between viewership of globally available and non-globally available titles?

#### Problem 4
You're a data scientist at Netflix.  Your boss tells you that they want to remove the quartile (25%) of titles that have the lowest viewership during this period -- it's just not worth the cost of hosting these titles!  Generate a list of these titles and write it to text file.

### Semi-meanies

#### Problem 5
Netflix dataset: Write a function that calulcates the ratio of non-Roman characters to total number of characters in a title, rounding this value to the nearest 0.01.  Apply it to the subset of the Netflix dataset that contains only series (*i.e.*, exclude movies), calling the new column `nr_ratio`.  Find the top-ten most-watched titles with `nr_ratio > 0.20`.

#### Problem 6
SDSS dataset: The basic spectrographic information in the SDSS dataset can actually tell us something about the masses of the galaxies therein.  That's crazy!  The `velDisp` feature in the data is a measure of distribution of velocities of the objects (stars) in a galaxy.  A larger `velDisp` (often given the symbol $\sigma$) means that there is a larger difference between the velocities of fastest and slowest objects.  This is actually an indirect measure of the mass of a galaxy (for complicated reasons).  Make a scatter plot of `velDisp` *vs* `redshift` (commonly called $\sigma$ *vs* $z$) for all of the non-broadband galaxies in the dataset.  Color each datapoint according to its subclass (don't forget a legend).  Keep in mind that (because of cosmological expansion) `redshift` is a measure of how far away an object is.  What relationship do you observe between these quantities?

### Meanie 1 -- Netflix Economist

The quantiles information for the Netflix data suggests that the megahours watched is unevenly distributed among titles.  A small number of titles get watched a LOT (*Let's Get Divorced: Season 1*???) and others barely at all.  We can investigate this by calculating the [*Gini coefficient*](https://en.wikipedia.org/wiki/Gini_coefficient) for the data.  (Note: This is a very common practice among macroeconomists who study inequality in many different systems!  Gini coefficient is a nice, general tool!)
Follow these steps to get your Gini coefficient.

- Create a dataframe that includes all non-series titles.  That is, omit any entry that has 'series' or 'season' in its title.  There are several ways to do this; you're welcome to steal from the examples above.  How many titles are in this dataframe?
- Create a new column that is `megahours_viewed` as we did in the examples above.
- Determine the maximum `megahours_viewed` for the titles in the dataframe.  Call this value $v_{max}$.
- Create a new column, `Mhv_ratio` that is each title's `megahours_viewed` divided by $v_{max}$.  Note that the maximum value of `Mhv_ratio` should be 1 and the minimum value should be close to zero.
- Make a histogram of `Mhv_ratio`.  Your histogram should have at least 100 bins and you should manually set the range on the horizontal axis to be 0 to 1.  Before you proceed to the next step, think carefully about what this histogram tells you.
- Now you want to scale your histogram so that the height of the tallest bar is 1.  Said another way, you want to divide the height of each bar by the height of the tallest bar.  Think back to our fitting lessons.  When you use `pyplot` to make a histogram, it outputs information about the bin edges and bar heights.  For example, you can get this information from a bit of code like
> `x, y, _ = plt.hist(mhv_ratio_array)`
- To this scaled histogram, add a straight line that connects the points $(0,0)$ and $(1,1)$.  Calculate the area under this line.  Call it $L$.  (Hint: It's a traingle.)
- Calculate the total area of all of the bars of your scaled histogram.  Call it $H$.  (Hint: Riemann sum.)
- Calculate the Gini coefficient: $G = (L-H)/L$.
- Do some googling to interpret the $G$ vaule you get.  Is it large compared to, say, inequalities in income, wealth, education, *etc.*?
- Congrats!  You're now an economist.  (Not really.)

### Meanie 2 -- Star Sleuth

In this problem you will investigate the subclasses of stars in the SDSS dataset.  Astronomers classify stars using the Morgan–Keenan (MK) system, in which each star falls into one of seven classes (called O, B, A, F, G, K, and M) according to its temperature.  A star's temperature is a function of the physics going on in it, so this is a useful classification scheme.  Check out the [Wikipedia article](https://en.wikipedia.org/wiki/Stellar_classification) for more info, most importantly
> Most stars are currently classified under the Morgan–Keenan (MK) system using the letters O, B, A, F, G, K, and M, a sequence from the hottest (O type) to the coolest (M type). Each letter class is then subdivided using a numeric digit with 0 being hottest and 9 being coolest (e.g., A8, A9, F0, and F1 form a sequence from hotter to cooler).

Let's see if the quantitative data we have contains information that correlates with the star classes (or whether the star classification was done by some other means).

- First, create a dataframe that contains only objects with STAR class and a subclass from the MK scheme described above.  For example, you'll need to omit subclasses such as 'M0V', 'L5.5', 'A0p', 'OB', 'WD', 'Carbon', *etc.*
- Create a new column in the dataframe called `MK_letter` which is just the first letter of the `subClass`.
- We can't directly determine the temperature of a star from our data, but the `spectroFlux_` features allow us indirect access.  Very simply the `spectroFlux_` features tell us how much light came from the object in particular parts of the spectrum.  For example, the `spectroFlux_r` quantity is a measure of how much of the object's light passes through SDSS's **red** filter.  By comparing several sepctroFluxes we can get an idea of the star's temperature.  (Much more information on this process is available [here](https://skyserver.sdss.org/dr1/en/proj/advanced/color/sdssfilters.asp) and on subsequent pages.)  Let's give each of these `spectroFlux_` quantities symbols like $F_g$, $F_r$, $F_u$, *etc.*  For each star in the dataframe, calculate the following relative flux differences:
$$
    f_{ug} = (F_u - F_g) / F_g
$$
$$
    f_{gr} = (F_g - F_r) / F_r \\    
$$
- Make a scatter plot of the $f_{ug}$ *vs* $f_{gr}$ for all of the stars in your dataframe.  Color each point according the its `MK_letter`.  Don't forget to include a legend!  You will want to make your data points slightly transparent so that you can see them well.
- Knowing what you know about the MK classification (*i.e.*, the temperature ordering of the classes) do you see a correlation between the relative flux differences and temperature?  